In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
unemp = pd.read_csv("/output.csv")

In [3]:
unemp.head()

,Year,Month,State,County,Rate
0,2015,February,Mississippi,Newton County,6.1
1,2015,February,Mississippi,Panola County,9.4
2,2015,February,Mississippi,Monroe County,7.9
3,2015,February,Mississippi,Hinds County,6.1
4,2015,February,Mississippi,Kemper County,10.6


In [4]:
#Now lets bring in our previously prepared Minimum wage dataset to see if we could combine these and get some relevant information
df = pd.read_csv("/minwage.csv")

In [5]:
df.head()

,Unnamed: 0,Year,State,Table_Data,Footnote,High.Value,Low.Value,CPI.Average,High.2018,Low.2018
0,0,1968,Alabama,...,NaN,0.00000,0.00000,34.783333,0.00,0.00
1,1,1968,Alaska,2.10,NaN,2.10000,2.10000,34.783333,15.12,15.12
2,2,1968,Arizona,18.72 - 26.40/wk(b),(b),0.66000,0.46800,34.783333,4.75,3.37
3,3,1968,Arkansas,1.25/day(b),(b),0.15625,0.15625,34.783333,1.12,1.12
4,4,1968,California,1.65(b),(b),1.65000,1.65000,34.783333,11.88,11.88


In [6]:
act_min_wage = pd.DataFrame()

for name, group in df.groupby("State"):
    if act_min_wage.empty:
        act_min_wage = group.set_index("Year")[["Low.2018"]].rename(columns={"Low.2018":name})
    else:
        act_min_wage = act_min_wage.join(group.set_index("Year")[["Low.2018"]].rename(columns={"Low.2018":name}))

act_min_wage.head()

,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,District of Columbia,Federal (FLSA),Florida,Georgia,Guam,Hawaii,Idaho,Illinois,Indiana,Iowa,Kansas,Kentucky,Louisiana,Maine,Maryland,Massachusetts,Michigan,Minnesota,Mississippi,Missouri,Montana,Nebraska,Nevada,New Hampshire,New Jersey,New Mexico,New York,North Carolina,North Dakota,Ohio,Oklahoma,Oregon,Pennsylvania,Puerto Rico,Rhode Island,South Carolina,South Dakota,Tennessee,Texas,U.S. Virgin Islands,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
Year,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1968,0.0,15.12,3.37,1.12,11.88,7.20,10.08,9.00,9.00,8.28,0.0,0.00,9.00,9.00,8.28,0.00,8.28,0.0,0.0,4.68,0.0,10.08,7.20,11.52,9.00,5.04,0.0,0.0,0.00,7.20,9.00,10.08,10.08,8.28,11.52,7.20,7.20,5.40,7.20,9.00,8.28,3.10,10.08,0.0,3.06,0.0,0.00,NaN,7.20,10.08,0.0,11.52,7.20,9.00,8.64
1969,0.0,14.33,3.19,1.07,11.26,6.83,9.56,8.53,8.53,7.85,0.0,0.00,8.53,8.53,7.85,0.00,7.85,0.0,0.0,4.44,0.0,9.56,6.83,10.92,8.53,4.78,0.0,0.0,0.00,6.83,8.53,9.56,9.56,7.85,10.92,6.83,6.83,5.12,6.83,8.53,7.85,2.94,9.56,0.0,2.90,0.0,0.00,NaN,6.83,9.56,0.0,10.92,6.83,8.53,8.19
1970,0.0,13.54,3.02,7.09,10.64,6.45,10.32,8.06,10.32,8.38,0.0,0.00,10.32,10.32,8.06,0.00,8.06,0.0,0.0,4.19,0.0,10.32,8.38,10.32,8.06,4.51,0.0,0.0,0.00,6.45,8.38,9.35,9.67,8.38,10.32,8.06,6.45,4.84,6.45,8.06,8.38,2.77,10.32,0.0,6.45,0.0,0.00,NaN,6.45,10.32,0.0,10.32,6.45,8.38,8.38
1971,0.0,12.99,2.89,6.80,10.20,6.18,9.89,7.73,9.89,8.04,0.0,0.00,9.89,9.89,7.73,0.00,7.73,0.0,0.0,4.02,0.0,9.89,8.04,9.89,7.73,4.33,0.0,0.0,0.00,6.18,8.04,8.97,9.28,8.04,9.89,7.73,6.18,4.64,6.18,7.73,8.04,2.66,9.89,0.0,6.18,0.0,0.00,NaN,6.18,9.89,0.0,9.89,6.18,8.04,8.04
1972,0.0,12.57,2.80,7.19,9.88,5.99,11.08,9.58,9.58,9.58,0.0,7.49,11.38,9.58,8.38,8.38,7.49,0.0,0.0,3.89,0.0,8.38,9.58,10.48,9.58,4.49,0.0,0.0,9.58,5.99,9.58,9.58,8.98,7.78,11.08,8.68,5.99,4.49,8.38,7.49,9.58,3.89,9.58,0.0,5.99,0.0,8.38,NaN,7.19,9.58,0.0,9.58,7.19,8.68,8.98


In [7]:
act_min_wage = act_min_wage.replace(0,np.NaN).dropna(axis=1)

In [8]:
#What do we need now? basically our purpose here is to check if there is any relationship between the 2 datasets i.e. are they correlated or not so this means that what we need is the correlation between the 2.
#For our correlation we need to create a 3rd dataset which will be basically the unemployment dataset with a new column of minimum wage added to it statewise so that we can correlate the both.
#lets define a function to ease our mapping.
def min_wage(year, state):
  try:
    return act_min_wage.loc[year][state]
  except:
    return np.NaN

In [9]:
min_wage(2007, "California")

9.06

In [10]:
#Looks like our function is working as required.Lets move ahead and do our mapping.
unemp['min_wage'] = list(map(min_wage, unemp['Year'], unemp['State']))

In [11]:
unemp.head()

,Year,Month,State,County,Rate,min_wage
0,2015,February,Mississippi,Newton County,6.1,NaN
1,2015,February,Mississippi,Panola County,9.4,NaN
2,2015,February,Mississippi,Monroe County,7.9,NaN
3,2015,February,Mississippi,Hinds County,6.1,NaN
4,2015,February,Mississippi,Kemper County,10.6,NaN


In [12]:
unemp.tail()

,Year,Month,State,County,Rate,min_wage
885543,2009,November,Maine,Somerset County,10.5,8.46
885544,2009,November,Maine,Oxford County,10.5,8.46
885545,2009,November,Maine,Knox County,7.5,8.46
885546,2009,November,Maine,Piscataquis County,11.3,8.46
885547,2009,November,Maine,Aroostook County,9.0,8.46


In [13]:
unemp[['Rate','min_wage']].corr()

,Rate,min_wage
Rate,1.000000,0.153047
min_wage,0.153047,1.000000


In [14]:
#we have a near to positive correlation between the unemployment rate and minimum wage.
#Lets try tp get their covariance too.
unemp[['Rate','min_wage']].cov()

,Rate,min_wage
Rate,9.687873,0.651586
min_wage,0.651586,1.874228


In [15]:
#There is a pretty strong covariance between them which implies that they do tend to vary together but the final impact of one on the other is not so much.
#lets try this new dataset with another one to move more deeper.
pres16 = pd.read_csv("/pres16results.csv")
#This is the US Election results dataset of 2016

In [16]:
pres16.head()

,county,fips,cand,st,pct_report,votes,total_votes,pct,lead
0,NaN,US,Donald Trump,US,0.9951,60350241.0,127592176.0,0.472993,Donald Trump
1,NaN,US,Hillary Clinton,US,0.9951,60981118.0,127592176.0,0.477938,Donald Trump
2,NaN,US,Gary Johnson,US,0.9951,4164589.0,127592176.0,0.032640,Donald Trump
3,NaN,US,Jill Stein,US,0.9951,1255968.0,127592176.0,0.009844,Donald Trump
4,NaN,US,Evan McMullin,US,0.9951,451636.0,127592176.0,0.003540,Donald Trump


In [17]:
top_candidates = pres16.head(10)['cand'].values

In [18]:
top_candidates

array(['Donald Trump', 'Hillary Clinton', 'Gary Johnson', 'Jill Stein',
       'Evan McMullin', 'Darrell Castle', 'Gloria La Riva',
       'Rocky De La Fuente', ' None of these candidates',
       'Richard Duncan'], dtype=object)

In [19]:
county_2015 = unemp[(unemp['Year']==2015)&(unemp['Month']=='February')] # Removing the data redundant to our cause.

In [20]:
county_2015.head()

,Year,Month,State,County,Rate,min_wage
0,2015,February,Mississippi,Newton County,6.1,NaN
1,2015,February,Mississippi,Panola County,9.4,NaN
2,2015,February,Mississippi,Monroe County,7.9,NaN
3,2015,February,Mississippi,Hinds County,6.1,NaN
4,2015,February,Mississippi,Kemper County,10.6,NaN


In [21]:
state_abbrev = pd.read_csv("/state_abbrev.csv", index_col = 0)#Importinh our previously created abbreviation dataset to use it again.

In [22]:
state_abbrev.head()

,State/District,Abbreviation,Postal Code
0,Alabama,Ala.,AL
1,Alaska,Alaska,AK
2,Arizona,Ariz.,AZ
3,Arkansas,Ark.,AR
4,California,Calif.,CA


In [23]:
state_abbrev = state_abbrev[['State/District','Postal Code']]

In [24]:
state_abbrev.set_index('State/District',inplace=True)

In [25]:
state_abbrev.head()

,Postal Code
State/District,
Alabama,AL
Alaska,AK
Arizona,AZ
Arkansas,AR
California,CA


In [26]:
state_abbrev_dict = state_abbrev.to_dict()['Postal Code']

In [27]:
state_abbrev_dict

{'Alabama': 'AL',
 'Alaska': 'AK',
 'Arizona': 'AZ',
 'Arkansas': 'AR',
 'California': 'CA',
 'Colorado': 'CO',
 'Connecticut': 'CT',
 'Delaware': 'DE',
 'District of Columbia': 'DC',
 'Florida': 'FL',
 'Georgia': 'GA',
 'Hawaii': 'HI',
 'Idaho': 'ID',
 'Illinois': 'IL',
 'Indiana': 'IN',
 'Iowa': 'IA',
 'Kansas': 'KS',
 'Kentucky': 'KY',
 'Louisiana': 'LA',
 'Maine': 'ME',
 'Maryland': 'MD',
 'Massachusetts': 'MA',
 'Michigan': 'MI',
 'Minnesota': 'MN',
 'Mississippi': 'MS',
 'Missouri': 'MO',
 'Montana': 'MT',
 'Nebraska': 'NE',
 'Nevada': 'NV',
 'New Hampshire': 'NH',
 'New Jersey': 'NJ',
 'New Mexico': 'NM',
 'New York': 'NY',
 'North Carolina': 'NC',
 'North Dakota': 'ND',
 'Ohio': 'OH',
 'Oklahoma': 'OK',
 'Oregon': 'OR',
 'Pennsylvania': 'PA',
 'Rhode Island': 'RI',
 'South Carolina': 'SC',
 'South Dakota': 'SD',
 'Tennessee': 'TN',
 'Texas': 'TX',
 'Utah': 'UT',
 'Vermont': 'VT',
 'Virginia': 'VA',
 'Washington': 'WA',
 'West Virginia': 'WV',
 'Wisconsin': 'WI',
 'Wyoming': 'WY

In [28]:
county_2015['State'] = county_2015['State'].map(state_abbrev_dict) #mapping the abbreviations with the county dataset this time.

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [29]:
len(county_2015)

2802

In [30]:
len(pres16)

18475

In [ ]:
#Since pres16 is longer we will map that to county_15.

In [31]:
pres16.rename(columns={'county': 'County', 'st':'State'}, inplace=True)#Since we are going to use .join to combine we need to index both pres and county by same Index which will be state and county here.

In [32]:
pres16.head()

,County,fips,cand,State,pct_report,votes,total_votes,pct,lead
0,NaN,US,Donald Trump,US,0.9951,60350241.0,127592176.0,0.472993,Donald Trump
1,NaN,US,Hillary Clinton,US,0.9951,60981118.0,127592176.0,0.477938,Donald Trump
2,NaN,US,Gary Johnson,US,0.9951,4164589.0,127592176.0,0.032640,Donald Trump
3,NaN,US,Jill Stein,US,0.9951,1255968.0,127592176.0,0.009844,Donald Trump
4,NaN,US,Evan McMullin,US,0.9951,451636.0,127592176.0,0.003540,Donald Trump


In [33]:
for i in [county_2015, pres16]:
    i.set_index(["County", "State"], inplace=True)

In [34]:
pres16 = pres16[pres16['cand']=='Donald Trump']
pres16 = pres16[['pct']]
pres16.dropna(inplace=True)


In [35]:
pres16.head()

pct
County State          
NaN    US     0.472993
       CA     0.330641
       FL     0.490640
       TX     0.525830
       NY     0.374752

In [36]:
county_2015.head()

,,Year,Month,Rate,min_wage
County,State,,,,
Newton County,MS,2015,February,6.1,NaN
Panola County,MS,2015,February,9.4,NaN
Monroe County,MS,2015,February,7.9,NaN
Hinds County,MS,2015,February,6.1,NaN
Kemper County,MS,2015,February,10.6,NaN


In [37]:
all_together = county_2015.merge(pres16, on=['County','State'])

In [38]:
all_together.dropna(inplace=True)
all_together.drop('Year',axis=1, inplace=True)

In [39]:
all_together.head()

,,Month,Rate,min_wage,pct
County,State,,,,
Major County,OK,February,2.6,2.11,0.864960
Pottawatomie County,OK,February,4.5,2.11,0.701342
Johnston County,OK,February,6.5,2.11,0.770057
Jefferson County,OK,February,5.0,2.11,0.812367
Beaver County,OK,February,2.8,2.11,0.888243


In [40]:
all_together.corr()

,Rate,min_wage,pct
Rate,1.000000,0.186689,-0.085985
min_wage,0.186689,1.000000,-0.325036
pct,-0.085985,-0.325036,1.000000


In [41]:
all_together.cov()

,Rate,min_wage,pct
Rate,5.743199,0.683870,-0.031771
min_wage,0.683870,2.336451,-0.076602
pct,-0.031771,-0.076602,0.023772


In [ ]:
#As we can see we have our final result here. min_wage has a negative correlation with the %vote for trump which implies that as the min_wage rises less people are going to vote for trump.
#Although they are correlated they do not appear to vary together.
#Maybe I have a little less knowledge on this subject and if somebody can throw in some few comments they are more than welcomed.